# メモ書き
- Mixed Presicion Trainingを使用するためには、以下2つのステップが必要。<br>
    1．計算上FP16で事が足りる箇所はFP16を適用する<br>
    2．勾配の値が小さくなっても保持するためにロススケーリングを使用する<br>
- より小さな精度で演算を行うメリットは以下の2つ<br>
    1．**メモリー使用量を削減**<br>
    例を挙げると、普段使用する精度はFP32(32ビット必要)。対してFP16(16ビット必要)。すなわちFP32からFP16に変更することで16ビッド(32ビット-16ビット)も削減できる。そうすることにより、より大きなモデル、またはより大きなミニバッチで学習させることができる。<br>
    2．**学習時間、または推論時間の短縮**<br>
    上の例で挙げたように、FP16を使用することにより、FP32を使用する時と比べて、アクセスするバイト数を半分(16ビット)にできる。そのため、計算時間も短縮できる。NVIDIAのGPUでは演算にFP16を使用すると、FP32で計算する時と比べて、最大8倍のスループットを実現する。